In [1]:
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn import model_selection
from sklearn.metrics import r2_score
import pandas as pd
import requests
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
from sklearn import linear_model as lm
from sklearn.metrics import accuracy_score
import requests
import time
from datetime import timedelta
from pandas import json_normalize
import json
import requests
from scipy.optimize import minimize
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import warnings


In [6]:
import fmp_python

from fmp_python.fmp import FMP

#this will pull be used to pull financial statements
fmp = FMP(output_format = 'pandas',api_key="2e1211f60d33c4daa1afec46de6d74f4")


In [7]:
#excel file with all of the tickers wee're using the data on
tickers = pd.read_excel('/Users/Lucas/Documents/Russell-3000-Stock-Tickers-List.xlsx')

#a few nameless/useless columns that were read with the tickers
tickers.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4','Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9','Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'],axis =1, inplace = True)

#make sure they're string formats
tick = tickers.astype(str)


#these are tickers that don't work anymore, need to be dropped
tick.drop([0,1,2,348,349,350,360,648,675,742,763,818,823,873,949,1002,1003,1004,1007,1133,1139,1178,1211,1263,1491,1559,1737,1765,2297,2333,2475,2659,2710,2732,2817,2850,2866,2867,2879,2932, 2934],axis = 0,inplace = True)#18

#turn the tick into a list
t = tick.iloc[:,0].tolist()



In [8]:
prices = []
ratios = []
cap = []
def pull2(i):
    #fmp module that gets price data
    x = fmp.get_historical_price(t[i])
    prices.append(x)
    #sleep to prevent too many api calls
    time.sleep(.2)
    html = "https://financialmodelingprep.com/api/v3/ratios/"+t[i]+"?apikey=2e1211f60d33c4daa1afec46de6d74f4"
    #request the html
    x = requests.get(html)
    #convert to json
    json = x.json()
    #convert to df
    df = json_normalize(json)
    #append to list
    ratios.append(df)
    #add wait period to not overload api above 300 calls per minute
    time.sleep(.2)
    html = "https://financialmodelingprep.com/api/v3/historical-market-capitalization/"+t[i]+"?apikey=2e1211f60d33c4daa1afec46de6d74f4"
    x = requests.get(html)
    json = x.json()
    df = json_normalize(json)
    time.sleep(.2)
    
    cap.append(df)


for i in range(len( (t))):
    pull2(i)

In [19]:
#this is for the initial gathering of data. Soemtimes the time runs out
#cuts the lists to equal lengths to run in the proceding code
prices = prices[:2475]
ratios = ratios[:2475]
cap = cap[:2475]

In [23]:
#get the data from the length of the lists already gather to the length of the list of all the tickers
for i in range(len(cap),len(t)):
    pull2(i)

In [9]:
print(len(prices))
print(len(ratios))
print(len(cap))

2921
2921
2921


In [10]:
#empty lists to get the good prices, ratios, and market caps. Some data that was pulled 
ratio2 = []
month_list = []
cap_list = []
#cleaning function
def clean(i): 
    if cap[i].index.size > 60:
        if prices[i].index.size>60:
             if ratios[i].index.size>1:   
                ratio2.append(ratios[i])
                month_list.append(prices[i])
                cap_list.append(cap[i])
    else:
        pass
for i in range(len(prices)):
    clean(i)

In [11]:
print(len(ratio2))
print(len(month_list))
print(len(cap_list))

2823
2823
2823


In [12]:
#this code gets the most recent market cap figures
#for whatever reason, the market cap records are incomplete, so I have to bootleg them
#bootlegging comes along later
todayCap = []
#function to get the most recent market caps
def skim(i):
    #get value of todays market cap
    x = cap_list[i][:1]['marketCap'].values
    #repeat the market cap for the length of each price
    m = np.repeat(x,len(month_list[i]))
    #turn into dataframe
    a = pd.DataFrame(data = m,columns =["marketCap"])
    #append to the already created list
    todayCap.append(a)
for i in range(len(cap_list)):
    skim(i)

    
#this code will get the 2nd neccesity for deriving cap, the adjclose
#eventually will use cap * (adjclose/constant)
multiply = []
def multi(i):
    #this gets todays adjusted close values
    x = month_list[i][:1]['adjClose'].values
    #intermediate array of todays adjusted close repeated by the length of the price data
    middle = np.repeat(x,len(month_list[i]))
    #turn it into a dataframe with the column name constant
    a = pd.DataFrame(data = middle,columns =["constant"])
    #append the dataframe to an already created list
    multiply.append(a)
    
for i in range(len(cap_list)):
    multi(i)

#list for getting all of the newly created month prices with constants and market caps    
new_month_list = []
def merge(i):
    #concatenate the prices, todays market caps, and constants together
    a = pd.concat([month_list[i],multiply[i],todayCap[i]],axis = 1)
    #append it to the list
    new_month_list.append(a)
for i in range(len(multiply)):
    merge(i)

In [13]:
#set the index to eventually cut by date
new_month_list = [x.set_index('date') for x in new_month_list]

In [14]:
#empty lists for chopped ratios
ra2021 = []
ra2020 = []
ra2019 = []
ra2018 = []
ra2017 = []
ra2016 = []

#split ratios by year
def chop2(i):
    r21 = ratio2[i][:1]
    r20 = ratio2[i][1:2]
    r19 = ratio2[i][2:3]
    r18 =ratio2[i][3:4]
    r17 =ratio2[i][4:5]
    r16 = ratio2[i][5:6]
    ra2021.append(r21)
    ra2020.append(r20)
    ra2019.append(r19)
    ra2018.append(r18)
    ra2017.append(r17)
    ra2016.append(r16)

for i in range(len(ratio2)):
    chop2(i)

entireRatios = [ra2017,ra2018,ra2019,ra2020,ra2021]

In [15]:
cutDates = []
def dateCapture(i):
    dates = []
    for x in range(len(entireRatios[i])):
        if len(entireRatios[i][x]) == 0:
            dates.append([])
        else:
            x = entireRatios[i][x]['date']
            dates.append(x)
    cutDates.append(dates)

In [16]:
for i in range(len(entireRatios)):
    dateCapture(i)

In [17]:
#empty lists
pr2017 = []
pr2018 = []
pr2019 = []
pr2020 = []
pr2021 = []

In [18]:
def chop(index1,appList,index2):
    prices = []
    for i in range(len(cutDates[index1])):
        if len(cutDates[index1][i]) != 0:
            if index1 == 4:
                x = new_month_list[i][:cutDates[index1][i][index2]]
                appList.append(x)
            else:
                x = new_month_list[i][cutDates[(index1+1)][i][(index2-1)]:cutDates[index1][i][index2]]
                appList.append(x)
        else:
            pass
    

In [19]:
chop(4,pr2021,0)

chop(3,pr2020,1)
chop(2,pr2019,2)
chop(1,pr2018,3)
chop(0,pr2017,4)

In [20]:
#turn into lists for ease of running
year_ratios = []
year_prices = [pr2017,pr2018,pr2019,pr2020,pr2021]

In [21]:
for x in entireRatios:
    g = []
    for i in x:
        if len(i) == 0:
            pass
        else:
            g.append(i)
    year_ratios.append(g)

In [22]:
for i in year_ratios:
    print(len(i))
print("\n")    
for i in year_prices:
    print(len(i))

2516
2625
2734
2823
2823


2516
2625
2734
2823
2823


In [23]:
#clean the prices and years to account for a potential lack of records
#in easier language, not every firm has ratios that go back 5 years, so this fixes the problem
#creates empy dataframes that mess up data running
c_year_prices = []
c_ratio = []
tickers = []
def clean2(index):
    need1 = []
    need2 = []
    for i in range(len(year_prices[index])):
        if year_ratios[index][i].index.size>0:
            if year_prices[index][i].index.size>1:
                need1.append(year_ratios[index][i])
                need2.append(year_prices[index][i])
        else:
            pass
    c_ratio.append(need1)
    c_year_prices.append(need2)
    

In [24]:
for i in range(len(year_prices)):
    clean2(i)

In [25]:
for x in c_year_prices:
    print(len(x))
for x in c_ratio:
    print(len(x))

2482
2607
2724
2805
2822
2482
2607
2724
2805
2822


In [26]:
#reverse dataframes to make them go in chronological order
c_prices = [[x[::-1] for x in x] for x in c_year_prices]

In [27]:
for x in c_prices:
    for x in x:
        x.reset_index(drop = True, inplace = True)

In [29]:
#calculate returns
import warnings
warnings.filterwarnings('ignore')
for x in c_prices:
    for x in x:
        x['returns'] = (x['close']-x['close'].shift(1))/x['close'].shift(1)
#fully bootstrap the market cap
for x in c_prices:
    for x in x:
        x['Mcap'] = (x['close']/x['constant']) * x['marketCap']
        x.drop(['constant','marketCap'],axis = 1,inplace = True)

In [326]:
#fill the nas
for x in c_prices:
    for x in x:
        x.fillna(0)

In [327]:
#this will be used for weighting later on by various matrics 
for x in c_prices:
    for x in x:
        x['total return'] = x['returns'] + 1
        x['weight'] = x['total return'] * (x['close']/(x['close'][:1].values))

In [33]:
c_prices[0][0]['Mcap'][(len(c_prices[0][0])-1):]

48    1.388204e+10
Name: Mcap, dtype: float64

In [328]:
caps = []
def cap(i):
    year = []
    for a in range(len(c_prices[i])):
        x = c_prices[i][a]['Mcap'][(len(c_prices[i][a])-1):]
        year.append(x)
    caps.append(year)
for i in range(len(c_prices)):
    cap(i)

In [39]:
caps[0]

[48    1.388204e+10
 Name: Mcap, dtype: float64,
 48    5.243238e+09
 Name: Mcap, dtype: float64,
 48    2.994617e+10
 Name: Mcap, dtype: float64,
 48    6.296477e+08
 Name: Mcap, dtype: float64,
 48    1.732983e+09
 Name: Mcap, dtype: float64,
 48    1.106703e+10
 Name: Mcap, dtype: float64,
 48    4.786319e+11
 Name: Mcap, dtype: float64,
 48    2.605147e+09
 Name: Mcap, dtype: float64,
 48    1.512809e+09
 Name: Mcap, dtype: float64,
 48    1.106007e+11
 Name: Mcap, dtype: float64,
 48    1.621747e+10
 Name: Mcap, dtype: float64,
 48    3.040289e+09
 Name: Mcap, dtype: float64,
 48    484479614.4
 Name: Mcap, dtype: float64,
 48    1.193235e+09
 Name: Mcap, dtype: float64,
 48    2.741642e+09
 Name: Mcap, dtype: float64,
 48    5.103953e+09
 Name: Mcap, dtype: float64,
 48    1.061289e+09
 Name: Mcap, dtype: float64,
 48    6.824765e+10
 Name: Mcap, dtype: float64,
 48    7.296155e+08
 Name: Mcap, dtype: float64,
 48    1.032284e+08
 Name: Mcap, dtype: float64,
 48    386727900.0
 N

In [329]:
caps2 = [np.array(pd.concat(x)) for x in caps]

In [330]:
#put all of the ratios into one dataframe, will need this for filtering later
yearlyR = [pd.concat(x) for x in c_ratio]

In [331]:
#reset index, again important
for x in yearlyR:
    x.reset_index(drop = True,inplace = True)

In [332]:

#need 
#ROA
#Operating CF/assets
#P/E ratio
#P/CFO ratio
#debt ratio
#grossProfitMargin
#assetturnover

In [333]:
#used for printing out the model performance, really adds to its cleanliness
years = ["2017 returns: ","2018 returns: ","2019 returns: ","2020 returns: ","2021 returns: "]

In [334]:
hprs = []
#this will calculate hodling period returns, useful for 2 resons
def hpr(i):
    r = []
    #get the sub list in c_prices
    for x in range(len(c_prices[i])):
        #price at time 0
        p0 = c_prices[i][x]['adjClose'][:1].values
        #price at time 1
        p1 = c_prices[i][x]['adjClose'][(len(c_prices[i][x])-1):].values
        #holding period return
        hpr = (p1-p0)/p0
        r.append(hpr)
    hprs.append(r)
for i in range(len(c_prices)):
    hpr(i)

In [337]:
for x in c_ratio:
    print(len(x))

2468
2588
2709
2796
2813


In [338]:
for x in hprs:
    print(len(x))

2468
2588
2709
2796
2813


This code will sort and pick out the chosen stocks

In [339]:
#make the HPR into yearly dataframes
con = [pd.DataFrame(i) for i in hprs]

In [340]:
#chnge column names
for x in con:
    x.columns = (["returns"])

In [341]:
#put all the ratios together
c_ratios = [pd.concat(x) for x in c_ratio]

In [342]:
for x in c_ratios:
    x.reset_index(drop = True, inplace = True)

In [343]:
ratios = []
#marge the returns and ratios together
def mergeR(i):
    x = pd.concat([c_ratios[i],con[i]],axis = 1)
    ratios.append(x)
for i in range(len(c_ratios)):
    mergeR(i)

In [344]:
market = [.0635,.2113,-.0524,.3102,.2089,.1632]

In [345]:
for i in range(len(ratios)):
    #binary variable on beating the market
    ratios[i]['binary m'] = np.where(ratios[i]["returns"]>market[i],1,0)
    #Binary variable on stock going down
    ratios[i]['binary down'] = np.where(ratios[i]["returns"]<0,1,0)
    #binary variable on stock going up
    ratios[i]['binary up down'] = np.where(ratios[i]["returns"]>0,1,0)

In [346]:
#isolate all of the years together
binary_down = [x['binary down'] for x in ratios]
binary_market = [x['binary m'] for x in ratios]
binary_updown = [x['binary up down'] for x in ratios]

In [347]:
#concatenate all of the years together
Ydown = pd.concat(binary_down)
Ymark = pd.concat(binary_market)
#Yud = pd.concat(binary_updown)

In [348]:
#reset index
Ydown.reset_index(drop = True,inplace = True)
Ymark.reset_index(drop = True,inplace =  True)

In [349]:
#drop them from tdataframes
for x in ratios:
    x.drop(['binary down','binary m','binary up down','returns'],axis = 1,inplace = True)

In [350]:
for x in ratios:
    print(len(x))

2468
2588
2709
2796
2813


In [351]:
#get rid of the useless columns
xnew =  [x.iloc[0:len(x ),3:] for x in ratios]

In [352]:
#fillanas with 0
x = [x.fillna(0) for x in xnew]

In [353]:
from scipy.stats.mstats import winsorize
columns = x[0].columns

In [354]:
#empy list for windzorized data
wdata = []
#this function wil windzorize a dataframe
def windzorize(i):
    slices = []
    #transpose the dataframe to actually be able to run it
    tran = x[i].T
    def slic(i):
        #slice by row then turn into numpy array
        x = tran[i:(i+1)].values
        
        slices.append(x)
    for i in range(len(tran)):
        slic(i)
    fixed_data = []
    def wind(i):
        a = slices[i]
        #windozrise individual arrays
        winsorize(a, limits=[0.15, 0.85])
        #append to list
        fixed_data.append(a)
    for i in range(len(slices)):
        wind(i)
    test2 = []
    #there is an extra braket, this code removes it
    def trim(i):
        test = fixed_data[i]
        test1 = []
        for i in range(len(test)):
            for a in range(len(test[i])):
                x = test[i][a]
                test1.append(x)
        test2.append(test1)
    for i in range(len(fixed_data)):
        trim(i)
    data = dict(zip(columns,test2 ))

    df = pd.DataFrame(data )
    wdata.append(df)
for i in range(len(x)):
    windzorize(i)

In [355]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()


In [356]:
#scale the data
scaled = [scale.fit_transform(g) for g in wdata]

In [357]:
#needs to be transposed again to eventually turn into dataframe
inter = [x.T for x in scaled]
#turn into list to remove dataframe
scales = [x.tolist() for x in inter]

In [358]:
dfs = []
#this will make new dictionaries
for i in range(len(scaled)):
    x = dict(zip(columns,scales[i] ))
    dfs.append(x)

In [359]:
X = [pd.DataFrame(x) for x in dfs]

In [360]:
x = pd.concat(X)
x.reset_index(drop = True,inplace = True)

This code will test machine learning techniques on portfolio management, namely predicting stocks that beat the market based off of ratios

In [361]:
Ydown['up'] = np.where(Ydown>0,0,1)

In [362]:
up = Ydown['up']

In [363]:
Ydown

0                                                        0
1                                                        0
2                                                        1
3                                                        0
4                                                        0
                               ...                        
13370                                                    0
13371                                                    0
13372                                                    0
13373                                                    0
up       [1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...
Name: binary down, Length: 13375, dtype: object

In [364]:
Ydown.drop(['up'] ,inplace = True)

In [365]:
U = pd.DataFrame(up)

In [366]:
U.columns = (['up'])

In [368]:

#dataframe with all x variables and beating the market y variable
market = pd.concat([Ymark,x],axis =1)
up = pd.concat([U,x],axis = 1)
#ud = pd.concat([Yud,X],axis = 1)

In [369]:
up

,up,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,grossProfitMargin,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
0,1,0.069628,-0.009234,0.058058,-0.009035,0.008110,0.007054,-0.044569,0.089612,0.482063,...,-0.022983,0.003639,-0.013775,-0.014628,-0.014628,-0.022221,-0.022983,-0.178733,-0.022974,-0.018912
1,1,-0.176863,-0.089720,-0.082909,-0.015317,-0.024363,-0.040148,-0.036859,0.029380,-0.368197,...,-0.023204,-0.038814,-0.020065,-0.022585,-0.022585,-0.022222,-0.023204,-0.435105,0.219923,-0.037383
2,0,-0.223679,-0.094319,-0.101706,-0.020288,-0.052546,-0.079893,-0.054750,0.023960,-0.116913,...,-0.023213,-0.012046,-0.022406,-0.017176,-0.017176,-0.022222,-0.023213,-0.246497,-0.036537,0.004460
3,1,-0.040929,-0.055162,-0.047328,-0.004818,0.016947,0.026262,-0.033403,0.087550,0.047532,...,-0.023098,-0.009072,-0.003391,-0.016423,-0.016423,-0.022222,-0.023098,-0.435105,-0.025101,-0.026832
4,1,0.048273,-0.032088,-0.061878,-0.011300,-0.015843,-0.021751,-0.058530,0.087530,-0.012155,...,-0.022880,0.005934,-0.008512,-0.012618,-0.012618,-0.022221,-0.022880,-0.261972,-0.017485,0.020365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13369,0,-0.124462,-0.019037,-0.019022,-0.023884,-0.019811,-0.020049,-0.050777,-0.019035,0.430100,...,-0.028345,-0.018828,0.030520,0.027625,0.027625,0.018729,-0.028345,-0.032326,-0.018862,-0.024542
13370,1,-0.059086,-0.019035,-0.019012,-0.022936,-0.017308,-0.017415,-0.042140,-0.016612,0.665868,...,-0.028341,-0.018608,0.059437,0.041071,0.041071,0.020270,-0.028341,-0.029858,-0.018807,0.079551
13371,1,-0.071334,-0.019036,-0.019033,-0.026014,-0.019147,-0.019682,-0.054350,-0.018554,-0.109642,...,-0.028348,-0.018703,0.006307,0.009322,0.009322,0.019100,-0.028348,-0.032326,-0.018824,-0.032983
13372,1,-0.118126,-0.019037,-0.019031,-0.020345,-0.019811,-0.019558,-0.064230,-0.018133,0.390960,...,-0.028344,-0.018832,-1.488447,0.193634,0.193634,0.018323,-0.028344,-0.032326,-0.018854,0.021186


In [371]:
#differentiate between majority and minority y variaables
up_majority = up[up['up']==1]
up_minority =up[up['up']==0]
#code to resample
up_upsampled = resample(up_minority,replace = True,n_samples = len(up_majority),random_state = 123)

In [372]:
#yet another repeat
market_majority = market[market['binary m']==0]
market_minority =market[market['binary m']==1]
market_upsampled = resample(market_minority,replace = True,n_samples = len(market_majority),random_state = 123)

In [373]:
#bring the resampled minority and majority back together
M1 = pd.concat([up_majority,up_upsampled])
M2 = pd.concat([market_majority,market_upsampled])

In [374]:
#get the Y variables for both
Y1 = M1['up'].values
Y2 = M2['binary m'].values

In [375]:
#drop the Y variables for both
M1.drop(['up'],axis = 1,inplace = True)
M2.drop(['binary m'],axis =1, inplace = True)

In [376]:

#scale = StandardScaler()
#scaled_data = scale.fit_transform(M)
#run pca
model = PCA(n_components = 5)
x = model.fit_transform(M1)
#change X variables into numpy arrays
x1 = M1.values
X1 = x1.astype(np.float32)
x2 = M2.values
X2 = x2.astype(np.float32)

In [377]:
#split into training data and test data
X_train1, X_test1,y_train1,y_test1 =  model_selection.train_test_split( X1, Y1.ravel(), test_size = 0.75, random_state = 7 )
X_train2, X_test2,y_train2,y_test2 =  model_selection.train_test_split( X2, Y2.ravel(), test_size = 0.75, random_state = 7 )

In [378]:
from sklearn.tree import DecisionTreeClassifier
#1 ia up down
modelG = DecisionTreeClassifier( criterion = 'gini', min_samples_split = 400,max_depth = 7,min_samples_leaf = 150,max_leaf_nodes=11)
modelG.fit(X_train1,y_train1)
predictions = modelG.predict(X_test1)
predictions2 = modelG.predict(X_train1)
print(accuracy_score(y_test1,predictions))
print(accuracy_score(y_train1,predictions2))
print( confusion_matrix( y_test1, predictions ) )

0.6473325310870437
0.6592057761732852
[[4454 1762]
 [2634 3615]]


In [379]:
from xgboost import XGBClassifier

model = XGBClassifier(gamma = 0, max_depth = 6,min_child_weight = 5)
model.fit(X_train1,y_train1)
predictions = model.predict(X_test1)
predictions2 = model.predict(X_train1)
a = (accuracy_score(y_test1, predictions))
b = accuracy_score(y_train1,predictions2)
print(a)
print(b)
print( confusion_matrix( y_test1, predictions ) )

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:31:04] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.7098275170477336
0.9843561973525873
[[4538 1678]
 [1939 4310]]


In [381]:
kfold = 10

results = model_selection.cross_val_score( model, X1, Y1, cv = kfold )

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:00:15] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:00:22] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:00:29] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

In [382]:
print(results.mean())

0.7231046931407942


In [132]:
from sklearn.model_selection import GridSearchCV
import warnings
params = {'min_child_weight' : range(0,10,1)}
warnings.filterwarnings('ignore')
gscv = GridSearchCV( estimator = XGBClassifier( max_depth = 6, gamma = 0,min_child_weight = 0), param_grid = params, n_jobs = 1, iid = False, cv = 5 )
gscv.fit( X_train1, y_train1.ravel() )

[19:50:22] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:50:25] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:50:27] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

[19:51:17] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:51:19] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:51:22] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

[19:52:11] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:52:14] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:52:16] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=0,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=6, min_child_weight=0,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate

In [137]:
predictions = gscv.predict(X_test1)
print(accuracy_score(predictions,y_test1))

0.7298280423280423


In [136]:
gscv.best_params_

{'min_child_weight': 5}

In [105]:
o9-from sklearn import svm
#this is for beating the market
model2 = svm.SVC( kernel = 'rbf', C = .2, gamma = 10 )
model2.fit( X_train2, y_train2 )
predictions = model2.predict(X_test2)
predictions2 = model2.predict(X_train2)
print(accuracy_score(y_test2,predictions))
print(accuracy_score(y_train2,predictions2))
print( confusion_matrix( y_test2, predictions ) )

0.6508799758289902
0.9546793564468615
[[5633  993]
 [3629 2984]]


This will be the more rigourous test where there is 0 data from the current year within the testing.